# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "cyberagent/calm2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = """USER: {input}\nASSISTANT: """
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.8,
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

1. 仕事の目標や目的を再確認する: 自分が何を目指して仕事をしているのか、その目標や目的を再度認識することで、自分の意欲を再燃させることができます。

2. チームワークを強化する: チームメンバーが互いに協力し、サポートし合うことで、チーム全体としての仕事の熱意に火をつけることができます。

3. 仕事の意義を再評価する: 仕事の価値や意義を見つめ直し、自分がどのように社会に貢献しているのかを理解することで、仕事に対するモチベーションを再燃させることができます。

4. 上司や同僚と密にコミュニケーションを取る: 上司や同僚と定期的にコミュニケーションを取ることで、自分がどのような仕事をしているのか、どのようなことを期待されているのかを理解することができます。また、チームワークを強化し、仕事に対するフィードバックやアドバイスを得ることもできます。

5. ストレスを解消する: 仕事によるストレスはモチベーションの低下につながることがあります。趣味や休息の時間を確保し、ストレスを解消することで、仕事に対する熱意を再燃させることができます。
タイトル：クマとサフィールの友情

ある日、クマのアンドリューは海辺で遊んでいるところ、遠くからサフィールというアザラシが近づいてきます。サフィールは海辺の食べ物を探しに来たらしく、二人はすぐに友達になりました。

アンドリューはサフィールと一緒に過ごす時間が楽しく、サフィールが海で泳ぐ様子や、魚を捕まえる様子を見て、アンドリューの心を満たしました。二人は楽しい時間を共有し、お互いの性格を知り合います。

アンドリューはサフィールが海辺で暮らす生活に満足しているように見えましたが、彼自身も自分が過ごす時間を楽しんでいました。しかし、ある日、サフィールが家に戻るための旅に出ることを決意しました。

アンドリューは寂しさを感じつつも、サフィールの旅を祈りました。そして、サフィールが戻ってきたとき、彼は彼の帰還を喜びました。

サフィールはアンドリューと過ごす時間がとても素晴らしいものであったと語りました。サフィールにとって、海辺で過ごす時間は彼にとってとても重要で、アンドリューとの友情はそれ以上に意味深いものとなりました。

この物語は、友情と持続可能性がテーマであり、サフィールとアンドリューの関係を通じて、異なる生活状況にある二人